Import library

In [16]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

# Read dataset

In [17]:
df_user_item = pd.read_csv('./dataset/user_rating.csv')
df_user_item.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,5.0,0.0,2.0,3.0,5.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,5.0,0.0,5.0,0.0,1.0,0.0,5.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


convert dataframe to numpy matrix

In [18]:
matrix = df_user_item.to_numpy()
matrix

array([[0., 3., 0., ..., 0., 0., 0.],
       [0., 3., 5., ..., 0., 0., 0.],
       [4., 5., 0., ..., 0., 0., 0.],
       ...,
       [0., 3., 0., ..., 0., 0., 0.],
       [4., 4., 5., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

# Rekomendasi

## 1. Average

### Group Recommendation using Average Method

The average method for group recommendations works by:
1. Taking individual ratings/preferences from each group member
2. Calculating the average rating for each item across all group members
3. Recommending items with the highest average ratings

In [19]:
def recommend_group_average(matrix, group_members, n_recommendations=5):
    # Get ratings for group members
    group_ratings = matrix[group_members]

    # Calculate average rating for each item
    average_ratings = np.mean(group_ratings, axis=0)

    # Get indices of top N items
    top_n_indices = np.argsort(average_ratings)[::-1][:n_recommendations]

    return top_n_indices, average_ratings[top_n_indices]

In [20]:
# Example usage
group = [0, 1, 2]  # Example group with users 0,1,2
recommended_items, scores = recommend_group_average(matrix, group)

print("Recommended items (indices):", recommended_items)
print("Recommendation scores:", scores)

Recommended items (indices): [40  7 12 36 34]
Recommendation scores: [5.         5.         4.66666667 4.66666667 4.66666667]


## 2. Least Misery

### Group Recommendation using Least Misery Method

The Least Misery strategy aims to minimize the disagreement within a group by:
1. Taking individual ratings from each group member
2. For each item, using the minimum rating across all group members
3. Recommending items with the highest minimum ratings

This ensures no group member strongly dislikes the recommended items.

In [21]:
def recommend_group_least_misery(matrix, group_members, n_recommendations=5):
    # Get ratings for group members
    group_ratings = matrix[group_members]

    # Calculate minimum rating for each item
    min_ratings = np.min(group_ratings, axis=0)

    # Get indices of top N items
    top_n_indices = np.argsort(min_ratings)[::-1][:n_recommendations]

    return top_n_indices, min_ratings[top_n_indices]

In [22]:
# Example usage with the same group
group = [0, 1, 2]  # Group with users 0,1,2
recommended_items, scores = recommend_group_least_misery(matrix, group)

print("Recommended items (indices):", recommended_items)
print("Recommendation scores:", scores)

Recommended items (indices): [ 7 40 12 34 36]
Recommendation scores: [5. 5. 4. 4. 4.]


## 3. Most Pleasure

### Group Recommendation using Most Pleasure Method

The Most Pleasure strategy focuses on maximizing satisfaction by:
1. Taking individual ratings from each group member
2. For each item, using the maximum rating across all group members
3. Recommending items with the highest maximum ratings

This approach emphasizes the most positive experiences of group members.

In [23]:
def recommend_group_most_pleasure(matrix, group_members, n_recommendations=5):
    # Get ratings for group members
    group_ratings = matrix[group_members]

    # Calculate maximum rating for each item
    max_ratings = np.max(group_ratings, axis=0)

    # Get indices of top N items
    top_n_indices = np.argsort(max_ratings)[::-1][:n_recommendations]

    return top_n_indices, max_ratings[top_n_indices]

In [24]:
# Example usage with the same group
group = [0, 1, 2]  # Group with users 0,1,2
recommended_items, scores = recommend_group_most_pleasure(matrix, group)

print("Recommended items (indices):", recommended_items)
print("Recommendation scores:", scores)


Recommended items (indices): [49 34 36 40 72]
Recommendation scores: [5. 5. 5. 5. 5.]


## 4. Borda

### Group Recommendation using Borda Method

The Borda count method works by:
1. Converting each member's ratings into rankings
2. Assigning points based on rankings (N-rank points, where N is number of items)
3. Summing up points across all members
4. Recommending items with highest total points

In [25]:
def recommend_group_borda(matrix, group_members, n_recommendations=5):
    # Get ratings for group members
    group_ratings = matrix[group_members]

    # Convert ratings to rankings for each user
    n_items = matrix.shape[1]
    borda_scores = np.zeros(n_items)

    for user_ratings in group_ratings:
        # Get ranking of items (handle ties with 'min' method)
        ranking = n_items - rankdata(user_ratings, method='min')
        # Add Borda points
        borda_scores += ranking

    # Get indices of top N items
    top_n_indices = np.argsort(borda_scores)[::-1][:n_recommendations]

    return top_n_indices, borda_scores[top_n_indices]

In [26]:
# Example usage with the same group
group = [0, 1, 2]  # Group with users 0,1,2
recommended_items, scores = recommend_group_borda(matrix, group)

print("Recommended items (indices):", recommended_items)
print("Recommendation scores:", scores)


Recommended items (indices): [99 33 76 75 74]
Recommendation scores: [297. 297. 297. 297. 297.]
